# This file is used for exploring the data

In [1]:
import pandas as pd
import numpy as np
from matplotlib.pyplot import hist
import math
import sqlite3

In [2]:
# Read in the data
courses = pd.read_csv('data/courses.csv')
assessments = pd.read_csv('data/assessments.csv')
vle = pd.read_csv('data/vle.csv')
students = pd.read_csv('data/studentInfo.csv')
student_registration = pd.read_csv('data/studentRegistration.csv')
student_assessments = pd.read_csv('data/studentAssessment.csv')
student_vle = pd.read_csv('data/studentVle.csv')

con = sqlite3.connect("data/ou.db")
cur = con.cursor()

In [3]:
test_assessments = student_assessments.copy(deep=True)

def add_time_before_deadline():
    global test_assessments
    joined_assessments = pd.merge(test_assessments, assessments, on='id_assessment')
    test_assessments['time_before_deadline'] = joined_assessments['date_submitted'] - joined_assessments['date']
    
def drop_banked():
    global test_assessments
    joined_assessments = pd.merge(test_assessments, assessments, on='id_assessment')
    test_assessments['is_banked'] = joined_assessments['is_banked']
    test_assessments = test_assessments[student_assessments.is_banked == 0]
    
def add_assessment_info():
    global test_assessments
    joined_assessments = pd.merge(test_assessments, assessments, on='id_assessment')
    test_assessments['weight'] = joined_assessments['weight']
    test_assessments['TMA'] = joined_assessments['assessment_type'] == "TMA"
    test_assessments['Exam'] = joined_assessments['assessment_type'] == "Exam"
    test_assessments['CMA'] = joined_assessments['assessment_type'] == "CMA"

def add_previous_average():
    global test_assessments
    # Use a sqlite database to make some calculations a lot quicker
    cur.execute('''
    SELECT (SELECT AVG(score)
        FROM assessments
                 JOIN student_assessments ON assessments.id_assessment = assessments.id_assessment AND
                                             student_assessments.id_student = sa.id_student
        WHERE date < a.date
          AND code_module = a.code_module
          AND assessments.code_presentation = a.code_presentation)
FROM assessments a
         JOIN student_assessments sa ON a.id_assessment = sa.id_assessment;
         ''')
    test_assessments['prev_avg'] = pd.DataFrame(cur.fetchall(), columns=['avg'])['avg']

def add_student_info():
    global test_assessments
    test_assessments = pd.merge(test_assessments, students, on='id_student')
    
# One hot encoding for assessment type
add_time_before_deadline()
drop_banked()
add_assessment_info()
add_previous_average()
add_student_info()

test_assessments.head()

,id_assessment,id_student,date_submitted,is_banked,score,time_before_deadline,weight,TMA,Exam,CMA,...,code_presentation,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,1752,11391,18,0,78.0,-1.0,10.0,True,False,False,...,2013J,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,1753,11391,53,0,85.0,-1.0,20.0,True,False,False,...,2013J,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
2,1754,11391,115,0,80.0,-2.0,20.0,True,False,False,...,2013J,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
3,1755,11391,164,0,85.0,-2.0,20.0,True,False,False,...,2013J,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
4,1756,11391,212,0,82.0,-3.0,30.0,True,False,False,...,2013J,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass


In [4]:
# May require --NotebookApp.iopub_data_rate_limit=1.0e10 jupyter parameter
def find_clicks():
    global test_assessments
    cur.execute('''
    SELECT COUNT(*)                       AS days_worked_on_assessment,
       SUM(student_vle.sum_click)     AS total_clicks_for_assessment,
       MIN(student_vle.date) - a.date AS days_started_before_assessment,
       student_vle.id_student,
       student_vle.id_assessment
FROM student_vle
         LEFT JOIN assessments a ON student_vle.id_assessment = a.id_assessment
         LEFT JOIN student_assessments sa
                   ON student_vle.id_assessment = sa.id_assessment AND student_vle.id_student = sa.id_student
    WHERE student_vle.id_assessment IS NOT NULL
GROUP BY student_vle.id_assessment, student_vle.id_student''')
    info = pd.DataFrame(cur.fetchall(), columns=['days_worked_on_assessment', 'total_clicks_for_assessment', 'days_started_before_assessment', 'id_student', 'id_assessment'])
    print(len(info))
    test_assessments = pd.merge(test_assessments, info, on=['id_student', 'id_assessment'])
find_clicks()

141062


In [5]:
test_assessments = test_assessments.drop(["days_worked_on_assessment_y", "total_clicks_for_assessment_y", "days_started_before_assessment_y"], axis=1)
hist(test_assessments.days_worked_on_assessment_x, bins=100)
hist(test_assessments.total_clicks_for_assessment_x, bins=100)
hist(test_assessments.days_started_before_assessment_x, bins=100)
test_assessments

KeyError: "['days_worked_on_assessment_y' 'total_clicks_for_assessment_y'\n 'days_started_before_assessment_y'] not found in axis"

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PowerTransformer

test_assessments = test_assessments.dropna()
# 100 and 80 needs to be dealt with, too many of them
#test_assessments = test_assessments[test_assessments.score != 100]
#test_assessments = test_assessments[test_assessments.score != 80]
X = test_assessments
X = X.drop(["id_assessment", "id_student", "is_banked", "score", "date_submitted", "gender", "region", 'highest_education', 'imd_band', 'age_band', 'disability', 'final_result', 'code_module', 'code_presentation'], axis=1)

y = test_assessments.score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
print(len(X_train))
hist(y_test, bins=100)
X.head()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

models = [
    LinearRegression(),
    #KNeighborsClassifier(),
    #MLPClassifier(),
    RandomForestClassifier(random_state=0)
]

for model in models:
    model.fit(X_train, y_train)
    #print(model.coef_)
    y_prediction = model.predict(X_test)
    print(math.sqrt(mean_squared_error(y_test, y_prediction)))
    hist(y_prediction, bins=100)